In [2]:
pip install pyathena


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
from datetime import datetime

In [4]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [5]:
#Connection to daily_check (based on receiver)
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_299/2235117015.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [6]:
#Connection to daily_forex 
forex_table = 'last_daily_forex'
database_name= 'viamericas'
df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)

/tmp/ipykernel_299/3034134324.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)


In [7]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [8]:
df = df[df['date'] != '2020-12-31'] # Excluyo el 31-12-2020

In [9]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [10]:
df['country'].unique()

array(['EL SALVADOR', 'MEXICO', 'ECUADOR', 'UNITED STATES', 'COLOMBIA',
       'NIGERIA', 'HONDURAS', 'UNITED STATES (ATM & DEPOSITS)',
       'GUATEMALA', 'BRAZIL', 'DOMINICAN REPUBLIC', 'MALI', 'NICARAGUA',
       'LIBERIA', 'INDIA', 'PHILIPPINES', 'BOLIVIA', 'ARGENTINA', 'BENIN',
       'HAITI', 'PERU', 'SENEGAL', 'GHANA', 'SIERRA LEONE', 'CHILE',
       'SPAIN', 'BANGLADESH', 'ITALY', 'TOGO', 'INDONESIA', 'NEPAL',
       'GUINEA', 'GAMBIA, THE', 'GERMANY', 'BELGIUM', 'JAMAICA',
       'CAMBODIA', 'BURKINA FASO', 'CAMEROON', 'URUGUAY',
       "COTE D'IVOIRE (IVORY COAST)", 'THAILAND', 'MALAYSIA', 'PORTUGAL',
       'UNITED KINGDOM', 'SWITZERLAND', 'COSTA RICA', 'FIJI', 'UGANDA',
       'ZAMBIA', 'KENYA', 'SRI LANKA', 'NETHERLANDS', 'LITHUANIA',
       'FRANCE', 'AUSTRIA', 'QATAR', 'GABON', 'ETHIOPIA', 'IRELAND',
       'RWANDA', 'TANZANIA', 'CZECH REPUBLIC', 'ROMANIA',
       'CONGO, (CONGO Â– BRAZZAVILLE)', 'MAURITANIA', 'VIETNAM', 'NORWAY',
       'PAKISTAN', 'LUXEMBOURG', 'CHAD',

In [11]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA - CHEQUEAR!

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_299/3736126815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [12]:
# FOREX - Selecting columns & renaming
df_rates=df_rates.rename(columns={'day': 'date', 'max_feed_price': 'feed_price'})
df_rates=df_rates.loc[:,['date', 'feed_price', 'symbol']]

In [13]:
df_rates

,date,feed_price,symbol
0,2021-11-28,50.4975,USDPHP
1,2021-11-28,75.0530,USDINR
2,2021-11-28,5.6097,USDBRL
3,2021-11-28,7.7387,USDGTQ
4,2021-11-28,21.9143,USDMXN
...,...,...,...
5120,2023-02-12,54.6550,USDPHP
5121,2023-02-12,5.2153,USDBRL
5122,2023-02-12,82.6400,USDINR
5123,2023-02-12,18.6940,USDMXN


In [14]:
def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' > chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [15]:
start_date = '2021-01-01'
end_date = '2023-10-22'

rates = get_closing_prices(df_rates, start_date, end_date)

In [16]:
###WE OBTAIN THE LAGS FOR THE EXCHANGE RATE 

def generate_lag_and_variation(df, num_lags):
    # Create columns for each day's lag up to the defined maximum
    for i in range(1, num_lags + 1):
        col_name = f'rate_lag_{i}'
        # Shift the 'feed_price' column grouped by 'symbol'
        df[col_name] = df.groupby('symbol')['feed_price'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, num_lags):
        col_name = f'var_rate_lag_{i}'
        # Calculate the difference between consecutive delay columns
        df[col_name] = df[f'rate_lag_{i}'] - df[f'rate_lag_{i + 1}']
    
    return df

In [17]:
rates_number = 14
rates = generate_lag_and_variation(rates, rates_number)
rates['date'] = rates['date'].dt.date #Date format

In [18]:
rates

,symbol,date,feed_price,rate_lag_1,rate_lag_2,rate_lag_3,rate_lag_4,rate_lag_5,rate_lag_6,rate_lag_7,...,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,var_rate_lag_7,var_rate_lag_8,var_rate_lag_9,var_rate_lag_10,var_rate_lag_11,var_rate_lag_12,var_rate_lag_13
0,USDBRL,2021-01-01,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USDBRL,2021-01-02,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDBRL,2021-01-03,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDBRL,2021-01-04,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDBRL,2021-01-05,5.3244,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5120,USDPHP,2023-10-18,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,56.8185,...,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766,0.0307,-0.1480
5121,USDPHP,2023-10-19,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,...,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766,0.0307
5122,USDPHP,2023-10-20,56.7786,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,...,-0.0300,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766
5123,USDPHP,2023-10-21,56.7010,56.7786,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,...,-0.1085,-0.0300,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486


In [19]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [20]:
rates['country'] = rates['symbol'].map(rates_dict)

In [21]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5125 entries, 0 to 5124
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           5125 non-null   object 
 1   date             5125 non-null   object 
 2   feed_price       5125 non-null   float64
 3   rate_lag_1       5120 non-null   float64
 4   rate_lag_2       5115 non-null   float64
 5   rate_lag_3       5110 non-null   float64
 6   rate_lag_4       5105 non-null   float64
 7   rate_lag_5       5100 non-null   float64
 8   rate_lag_6       5095 non-null   float64
 9   rate_lag_7       5090 non-null   float64
 10  rate_lag_8       5085 non-null   float64
 11  rate_lag_9       5080 non-null   float64
 12  rate_lag_10      5075 non-null   float64
 13  rate_lag_11      5070 non-null   float64
 14  rate_lag_12      5065 non-null   float64
 15  rate_lag_13      5060 non-null   float64
 16  rate_lag_14      5055 non-null   float64
 17  var_rate_lag_1

In [22]:
df1 = pd.merge(df_filtered, rates, on=['date', 'country'], how='left')
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [23]:
### EFFECT OF CANCELED TRANSACTIONS ###
database_name= 'analytics'
table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS

df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[df2['date'] != '2020-12-31'] # Elimino el registro del 31.12.2020

/tmp/ipykernel_299/1901381988.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)


In [24]:
from sklearn.preprocessing import LabelEncoder

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df2['payer_country'] = df2['payer'] + '_' + df2['country']
# Inicializing LabelEncoder
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df2['payer_country_encoder'] = label_encoder.fit_transform(df2['payer_country'])

In [25]:
# By applying the same aging filter, we can work on the same payer_country universe 
df2 = df2[df2['payer_country'].isin(df_aging['payer_country'])]

In [26]:
def fill_missing_dates(df, start_date, end_date):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=start_date, end=end_date)
    df_fill = pd.DataFrame({'date': date_range, 'amount': 0, 'tx_cancelled': 0})
    df_fill['date'] = pd.to_datetime(df_fill['date']).dt.date
    
    # Sort the original DataFrame by 'country', 'payer', and 'date'
    df = df.sort_values(by=['country', 'payer', 'date'])
    
    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'payer_country'
    for payer_country in df['payer_country'].unique():
#        print(payer_country)
        df_aux = df[df['payer_country'] == payer_country]
        # Combining df_aux (payer_country) with df_fill, keeping values from df_aux and filling dates, amount and tx_cancelled 
        merged_df = df_aux.set_index('date').combine_first(df_fill.set_index('date')).reset_index()

        # Fill missing values in 'payer', 'country', 'payer_country', 'payer_country_encoder' columns
        columns_to_fill = ['payer', 'country', 'payer_country', 'payer_country_encoder']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    
    return result_df

In [27]:
#start_date and end_date variables defined previously
start_date = '2021-01-01'
end_date = '2023-10-21'

# Call the function with the specified start_date and end_date
df_full = fill_missing_dates(df2, start_date, end_date)

In [28]:
def generate_tx_lags_and_variation(df, tx_count):
    """
    Generate lag columns for cancelled transactions and their variations.

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_cancelled_lag_{i}'
        # Shift the 'tx_cancelled' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx_cancelled'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, tx_count):
        col_name = f'var_tx_cancelled_lag_{i}'
        # Calculate the difference between consecutive lag columns
        df[col_name] = df[f'tx_cancelled_lag_{i}'] - df[f'tx_cancelled_lag_{i + 1}']

    return df


In [29]:
# Call the function and assign the result back to df2
tx_cancelled_lags = 7
df2 = generate_tx_lags_and_variation(df_full, tx_cancelled_lags)

In [30]:
df2['date'] = df2['date'].dt.date

In [31]:
df2

,date,amount,country,payer,payer_country,payer_country_encoder,tx_cancelled,tx_cancelled_lag_1,tx_cancelled_lag_2,tx_cancelled_lag_3,tx_cancelled_lag_4,tx_cancelled_lag_5,tx_cancelled_lag_6,tx_cancelled_lag_7,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6
0,2021-01-01,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,25.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,2021-01-05,490.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133115,2023-10-17,0.0,UNITED STATES (ATM & DEPOSITS),VIAMERICAS USA DEPOSITS,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,287.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133116,2023-10-18,0.0,UNITED STATES (ATM & DEPOSITS),VIAMERICAS USA DEPOSITS,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,287.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133117,2023-10-19,0.0,UNITED STATES (ATM & DEPOSITS),VIAMERICAS USA DEPOSITS,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,287.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133118,2023-10-20,0.0,UNITED STATES (ATM & DEPOSITS),VIAMERICAS USA DEPOSITS,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,287.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'], how='outer')
df_final['date'] = pd.to_datetime(df_final['date'])

In [33]:
df_final.columns

Index(['payer', 'country', 'date', 'tx', 'amount', 'coupon_count',
       'payer_country_x', 'symbol', 'feed_price', 'rate_lag_1', 'rate_lag_2',
       'rate_lag_3', 'rate_lag_4', 'rate_lag_5', 'rate_lag_6', 'rate_lag_7',
       'rate_lag_8', 'rate_lag_9', 'rate_lag_10', 'rate_lag_11', 'rate_lag_12',
       'rate_lag_13', 'rate_lag_14', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6',
       'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
       'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
       'payer_country_y', 'payer_country_encoder', 'tx_cancelled',
       'tx_cancelled_lag_1', 'tx_cancelled_lag_2', 'tx_cancelled_lag_3',
       'tx_cancelled_lag_4', 'tx_cancelled_lag_5', 'tx_cancelled_lag_6',
       'tx_cancelled_lag_7', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_c

In [34]:
###WE SHOULD KEEP THE VARIABLES THAT HAVE IMPUTED MISSINGS IN AMOUNT AND TX_CANCELLED (DAYS WITHOUT OPERATIONS) BUT KEEP THE MISSINGS IN RATES
df_final=df_final.loc[:,['date', 'payer', 'country', 'amount', 
                         'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3',
                         'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7',
                         'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
                         'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
                         'payer_country_encoder','payer_country_y',
                         'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
                         'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6']]
df_final = df_final.rename(columns={'payer_country_y': 'payer_country'})

In [35]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    133120 non-null  datetime64[ns]
 1   payer                   133120 non-null  object        
 2   country                 133120 non-null  object        
 3   amount                  133120 non-null  float64       
 4   var_rate_lag_1          33730 non-null   float64       
 5   var_rate_lag_2          33701 non-null   float64       
 6   var_rate_lag_3          33673 non-null   float64       
 7   var_rate_lag_4          33644 non-null   float64       
 8   var_rate_lag_5          33617 non-null   float64       
 9   var_rate_lag_6          33589 non-null   float64       
 10  var_rate_lag_7          33561 non-null   float64       
 11  var_rate_lag_8          33534 non-null   float64       
 12  var_rate_lag_9          33505 

In [36]:
### DUMMIES ####

# Weekend indicator (dummy)
df_final['weekend'] = (df_final['date'].dt.dayofweek >= 4).astype(int)

In [37]:
#WE ADD ONE MORE DAY AFTER CHRISTMAS, NEW YEAR'S EVE AND US INDEPENDENCE (BETTER CAPTURES THE DISTURBANCE).
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26',
                     '07-05'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [38]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [39]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [40]:
df_final['amount'].sum()

24303285928.466

In [41]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    133120 non-null  datetime64[ns]
 1   payer                   133120 non-null  object        
 2   country                 133120 non-null  object        
 3   amount                  133120 non-null  float64       
 4   var_rate_lag_1          33730 non-null   float64       
 5   var_rate_lag_2          33701 non-null   float64       
 6   var_rate_lag_3          33673 non-null   float64       
 7   var_rate_lag_4          33644 non-null   float64       
 8   var_rate_lag_5          33617 non-null   float64       
 9   var_rate_lag_6          33589 non-null   float64       
 10  var_rate_lag_7          33561 non-null   float64       
 11  var_rate_lag_8          33534 non-null   float64       
 12  var_rate_lag_9          33505 

In [42]:
# Impute missings in exogenous variables
df_final.fillna(0, inplace=True)

In [43]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    133120 non-null  datetime64[ns]
 1   payer                   133120 non-null  object        
 2   country                 133120 non-null  object        
 3   amount                  133120 non-null  float64       
 4   var_rate_lag_1          133120 non-null  float64       
 5   var_rate_lag_2          133120 non-null  float64       
 6   var_rate_lag_3          133120 non-null  float64       
 7   var_rate_lag_4          133120 non-null  float64       
 8   var_rate_lag_5          133120 non-null  float64       
 9   var_rate_lag_6          133120 non-null  float64       
 10  var_rate_lag_7          133120 non-null  float64       
 11  var_rate_lag_8          133120 non-null  float64       
 12  var_rate_lag_9          133120

In [44]:
##mantenemos mismos id_payer que antes##
del df_final['payer_country_encoder']
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df_final['payer_country_encoder'] = label_encoder.fit_transform(df_final['payer_country'])

In [45]:
# Setting DS timeframe

df_june23 = df_final.loc[df_final.date<'2023-07-01']
df_20_12 = df_final.loc[df_final.date<'2022-12-21']
df_29_12 = df_final.loc[df_final.date<'2022-12-30']
df_05_05 = df_final.loc[df_final.date<'2023-05-06']
df_03_20 = df_final.loc[df_final.date<'2023-03-21']
df_09_10 = df_final.loc[df_final.date<'2023-09-11']
df_11_25 = df_final.loc[df_final.date<'2022-11-26']
df_02_04 = df_final.loc[df_final.date<'2023-02-05']

In [46]:
df_june23['amount'].sum()

20321812549.673996

In [47]:
df_june23.columns

Index(['date', 'payer', 'country', 'amount', 'var_rate_lag_1',
       'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9',
       'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'payer_country', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_cancelled_lag_6', 'weekend', 'special_dates',
       'payer_country_encoder'],
      dtype='object')

In [48]:
##Top 10 payers##
#total_amount_per_payer = df_final.groupby('payer_country_encoder')['amount'].sum()
#top_10_payers = total_amount_per_payer.sort_values(ascending=False).head(10)
#print(top_10_payers)

In [49]:
#top_10_payers=[57,29,36,24,38,60,32,20,19,101]
#top_10 = df_final[df_final['payer_country_encoder'].isin(top_10_payers)]

In [50]:
#top_10.country.unique()

In [51]:
#top_10.loc[top_10.country=='HONDURAS']

In [52]:
df_june23

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
0,2021-06-17,BANCO AGRICOLA,EL SALVADOR,198615.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,BANCO AGRICOLA_EL SALVADOR,-5.0,-2.0,10.0,-7.0,-1.0,-4.0,0,0,11
1,2021-06-17,CHEDRAUI (APPRIZA),MEXICO,17128.00,0.3388,0.1163,0.0604,0.0178,0.0000,0.1222,...,CHEDRAUI (APPRIZA)_MEXICO,1.0,0.0,-2.0,0.0,2.0,-4.0,0,0,44
3,2022-08-16,BANCO DE GUAYAQUIL,ECUADOR,54357.65,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,BANCO DE GUAYAQUIL_ECUADOR,8.0,-3.0,-5.0,5.0,2.0,-1.0,0,0,18
4,2021-11-12,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,391240.62,0.0190,0.2748,-0.0418,0.0203,0.0363,-0.2537,...,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,4.0,6.0,2.0,-1.0,-3.0,-11.0,1,0,101
5,2023-06-21,VIAMERICAS USA DEPOSITS,UNITED STATES,95073.29,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,VIAMERICAS USA DEPOSITS_UNITED STATES,-2.0,3.0,-3.0,-2.0,1.0,8.0,0,0,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132940,2023-06-28,MASTERCARD ATM NETWORK,UNITED STATES (ATM & DEPOSITS),0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,MASTERCARD ATM NETWORK_UNITED STATES (ATM & DE...,-2.0,2.0,-1.0,1.0,-1.0,1.0,0,0,78
132941,2023-06-29,MASTERCARD ATM NETWORK,UNITED STATES (ATM & DEPOSITS),0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,MASTERCARD ATM NETWORK_UNITED STATES (ATM & DE...,2.0,-2.0,2.0,-1.0,1.0,-1.0,0,0,78
133039,2021-11-11,VIAMERICAS USA DEPOSITS,UNITED STATES (ATM & DEPOSITS),0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,0.0,1.0,0.0,1.0,-3.0,3.0,0,0,122
133040,2022-01-01,VIAMERICAS USA DEPOSITS,UNITED STATES (ATM & DEPOSITS),0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,VIAMERICAS USA DEPOSITS_UNITED STATES (ATM & D...,0.0,0.0,0.0,-2.0,-1.0,4.0,1,0,122


In [54]:
control=df_final.loc[df_final.payer_country_encoder==57]

In [55]:
##Elektra volvió a ser el 57##
control

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
3714,2022-06-09,ELEKTRA (MEXICO),MEXICO,5.093494e+06,0.0246,0.0293,0.0190,0.0061,-0.0167,-0.1426,...,ELEKTRA (MEXICO)_MEXICO,-69.0,-93.0,-29.0,-30.0,118.0,97.0,0,0,57
3836,2021-09-24,ELEKTRA (MEXICO),MEXICO,5.734611e+06,-0.0392,-0.0223,0.0216,0.0552,0.0483,0.0000,...,ELEKTRA (MEXICO)_MEXICO,-45.0,-3.0,-104.0,72.0,5.0,90.0,1,0,57
4108,2022-07-16,ELEKTRA (MEXICO),MEXICO,8.815708e+06,-0.0756,0.1157,0.0151,0.0136,0.3118,0.0000,...,ELEKTRA (MEXICO)_MEXICO,-6.0,71.0,-94.0,-76.0,-49.0,56.0,1,0,57
4147,2021-05-04,ELEKTRA (MEXICO),MEXICO,3.853014e+06,0.0221,0.0016,0.0000,0.1919,-0.0139,0.0040,...,ELEKTRA (MEXICO)_MEXICO,12.0,-20.0,44.0,47.0,-17.0,7.0,0,0,57
4155,2021-10-31,ELEKTRA (MEXICO),MEXICO,6.187509e+06,0.0000,0.1564,0.0798,0.1096,0.0011,0.0177,...,ELEKTRA (MEXICO)_MEXICO,77.0,105.0,-108.0,84.0,-104.0,3.0,1,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87293,2021-07-27,ELEKTRA (MEXICO),MEXICO,3.555608e+06,0.0367,0.0153,-0.0633,-0.0617,-0.0073,0.0342,...,ELEKTRA (MEXICO)_MEXICO,44.0,-44.0,24.0,98.0,-11.0,-28.0,0,0,57
87304,2021-12-12,ELEKTRA (MEXICO),MEXICO,5.996044e+06,-0.1199,0.0019,-0.0248,-0.2420,0.0096,-0.0257,...,ELEKTRA (MEXICO)_MEXICO,29.0,100.0,-46.0,130.0,-187.0,-4.0,1,0,57
87450,2021-12-28,ELEKTRA (MEXICO),MEXICO,3.951326e+06,0.0759,0.0096,-0.0221,-0.0776,-0.1132,0.0063,...,ELEKTRA (MEXICO)_MEXICO,7.0,421.0,-500.0,-53.0,117.0,12.0,0,0,57
87520,2023-07-20,ELEKTRA (MEXICO),MEXICO,6.925819e+06,0.0110,-0.0653,0.0664,0.0068,-0.1508,-0.0272,...,ELEKTRA (MEXICO)_MEXICO,-59.0,-92.0,8.0,-51.0,126.0,97.0,0,0,57


In [58]:
# TARGET CSV
target_ts_amount=df_june23.loc[:, ['payer_country_encoder', 'date', 'amount']]
target_ts_amount.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp', 'amount': 'target_value'}, inplace=True)
target_ts_amount.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1024 entries, 3714 to 87616
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       1024 non-null   int64         
 1   timestamp     1024 non-null   datetime64[ns]
 2   target_value  1024 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 32.0 KB


In [ ]:
# RELATED TS
###GENERATE THE DATASET OF RELATED TIME SERIES (UP TO 23 VARIABLES+ID_ITEM+TIMESTAMP-FIRST THESE TWO VARIABLES)
related_ts_columns = ['payer_country_encoder', 'date', 
                      'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend', 'special_dates']

related_ts_ds = df_june23[related_ts_columns]
related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)

In [60]:
##ONLY FOUR EXOGENOUS VARIABLES FOR EXPLAINABILITY TEST
related_ts_columns = ['payer_country_encoder', 'date', 
                      'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4']

related_ts_ds = df_june23[related_ts_columns]
related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)

/tmp/ipykernel_299/2457490285.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)


In [61]:
related_ts_ds.columns

Index(['item_id', 'timestamp', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4'],
      dtype='object')

In [ ]:
 #[item_id, timestamp, var_rate_lag_1, var_rate_lag_2, var_rate_lag_3, var_rate_lag_4, var_rate_lag_5, var_rate_lag_6, var_rate_lag_7, var_rate_lag_8, var_rate_lag_9, var_rate_lag_10, var_rate_lag_11, var_rate_lag_12, var_rate_lag_13, var_tx_cancelled_lag_1, var_tx_cancelled_lag_2, var_tx_cancelled_lag_3, var_tx_cancelled_lag_4, var_tx_cancelled_lag_5, var_tx_cancelled_lag_6, weekend, special_dates]

In [58]:
###GENERATE A DATASET WITH METADATA(COUNTRY AND PAYER)
metadata_ts=df_june23.loc[:, ['payer_country_encoder', 'country', 'payer']]
metadata_ts.rename(columns={'payer_country_encoder': 'item_id'}, inplace=True)
metadata_ts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118430 entries, 0 to 133041
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   item_id  118430 non-null  int64 
 1   country  118430 non-null  object
 2   payer    118430 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


In [ ]:
pip install awswrangler

In [62]:
#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#
##ROUTINES FOR EXPORTING CSV TO VIAMERICAS S3##
import awswrangler as wr
file_name_target = 'target_abt_4lags_june23_with_zeros.csv'

wr.s3.to_csv(
    df=target_ts_amount,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/{file_name_target}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/target_abt_4lags_june23_with_zeros_elektra.csv'],
 'partitions_values': {}}

In [63]:
import awswrangler as wr
file_name_ts = 'related_ts_four_vars_june23_with_zeros.csv'

wr.s3.to_csv(
    df=related_ts_ds,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_ts}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/related_ts_four_vars_june23_with_zeros_elektra.csv'],
 'partitions_values': {}}

In [60]:
file_name_metadata = 'metadata_4lags_june23_with_zeros.csv'

wr.s3.to_csv(
    df=metadata_ts,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_metadata}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/metadata_4lags_june23_with_zeros.csv'],
 'partitions_values': {}}